<a href="https://colab.research.google.com/github/rona11-17/pytorch_tutorial/blob/main/0_Learn_the_Basics/Pytorch0_6_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch入門6.最適化

In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits


model = NeuralNetwork()


100%|██████████| 26.4M/26.4M [00:01<00:00, 17.6MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 296kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.58MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.7MB/s]


In [4]:
# ハイパーパラメータの設定
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 損失関数のセット
loss_fn = nn.CrossEntropyLoss()

# 最適化手法のセット
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

---
## 最適化ループ
各エポックは2種類のループから構成される
- 訓練ループ
- 検証/テストループ

訓練ループ内の最適化は3つのステップから構成される
1. optimizer.zero_gradでモデルパラメータの勾配をリセット
2. loss.backwards()を実行し、誤差逆伝播を実行
3. optimizer.step()を実行し、各パラメータの勾配を使用してパラメータを調整

---
## 実装全体

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # 予測と損失の計算
    pred = model(X)
    loss = loss_fn(pred, y)

    # バックプロぱげーション
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= size
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Abg loss: {test_loss:>8f} \n")

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epocks = 10
for t in range(epocks):
  print(f"Epock {t+1}\n-----------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epock 1
-----------------------
loss: 2.296608 [    0 / 60000]
loss: 2.291032 [ 6400 / 60000]
loss: 2.275388 [12800 / 60000]
loss: 2.272488 [19200 / 60000]
loss: 2.262043 [25600 / 60000]
loss: 2.238742 [32000 / 60000]
loss: 2.249823 [38400 / 60000]
loss: 2.217973 [44800 / 60000]
loss: 2.202534 [51200 / 60000]
loss: 2.201185 [57600 / 60000]
Test Error: 
 Accuracy: 44.6%, Abg loss: 0.034196 

Epock 2
-----------------------
loss: 2.182396 [    0 / 60000]
loss: 2.180436 [ 6400 / 60000]
loss: 2.123455 [12800 / 60000]
loss: 2.147833 [19200 / 60000]
loss: 2.102391 [25600 / 60000]
loss: 2.064610 [32000 / 60000]
loss: 2.110391 [38400 / 60000]
loss: 2.031811 [44800 / 60000]
loss: 2.027889 [51200 / 60000]
loss: 2.030252 [57600 / 60000]
Test Error: 
 Accuracy: 55.3%, Abg loss: 0.030961 

Epock 3
-----------------------
loss: 1.990966 [    0 / 60000]
loss: 1.975782 [ 6400 / 60000]
loss: 1.855312 [12800 / 60000]
loss: 1.930572 [19200 / 60000]
loss: 1.829816 [25600 / 60000]
loss: 1.783656 [32000 / 6